In [1]:
import json

# things we need for NLP
import string
import nltk
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
from nltk.corpus import stopwords

# things we need for Tensorflow
import numpy as np
import tensorflow as tf
import random


import keras
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense

# things we need for the conversation 
import emoji
from termcolor import colored
from gingerit.gingerit import GingerIt
import numpy as np

Using TensorFlow backend.


In [2]:
#Import training data
with open('new_intent.json', 'r') as f:
    intents = json.load(f)

In [3]:
#Autonomous community telephone numbers
with open('comunidades_autonomas.json') as o:
    ca = json.load(o)

### 1. NLP: preprocessing techniques

Now we will apply some preprocessing techniques to our training words. The set of all words in each of the patterns from every different tag constitute the training data. 

Since the user interacting with the chatbot is likely to introduce words different from and in a different sequence than those of our training data, we must select the most relevant words in the most identifiable format to compare them with the user input and be able to find a suitable response. 

#### 1.1 Tokenization

Tokenization is the process of splitting the given text into smaller pieces called tokens. Words, numbers, puntuation marks, and others can be considered as tokens

#### 1.2 Stemming
Stemming is a process of reducing words to their word stem, base or root form (for example, books = book, looked = look).


### Bag of words model

We will first define our vocabulary by preprocessing the training data with the methods described above and then, we will create vectors with 0's and 1's for each of the sentences with a length equal to the length of the vocabulary, where 0 will mean that the word is absent in the sentence and 1 means that the word is present in the sentence. 

#### 1. Vocabulary

In [4]:
def tokenize (sentence):
    return nltk.word_tokenize(sentence)

def stem(word):
    return stemmer.stem(word.lower())

def bag_of_words (tokenized_sentence, vocabulary):
    
    """
    sentence = ["hello", "how", "are", "you"]
    vocabulary = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
    bag        = [  0,     1,     0,    1,     0,      0,       0  ]
    
    """
    #Stem the sentence
    sentence = [stem(w) for w in tokenized_sentence]
    
    #All-zeros array for bag
    bag = np.zeros(len(vocabulary))
    
    '''
    For each word in the vocabulary, if that word is present in the sentence,
    put a 1 value in that position correspondant to the position of the word
    in the vocabulary list, 0 otherwise. 
    '''
    for idx, w in enumerate(vocabulary):
        if w in sentence:
            bag[idx] = 1
    
    return bag


In [5]:
def bag_nostem(tokenized_sentence, vocabulary):
    
    tokenized_sentence = [w.lower() for w in tokenized_sentence]
    
    bag = np.zeros(len(vocabulary))
    
    for idx, w in enumerate(vocabulary):
        if w in tokenized_sentence:
            bag[idx] = 1
    
    return bag

In [6]:
#Create an empty list with pattern words 
vocabulary = []
#Create an empty list with tag words 
tags = []
#Create an empty list with tag and pattern words 
xy = []
#Create an empty list with tag and tag index 
tag_idx = []
#Create list with responses
responses = []

for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    response_list = []
    
    for pattern in intent['patterns']:
        #Tokenize each word in the sentence
        words = tokenize(pattern)        
        #Add words to the pattern_words list
        vocabulary.extend(words)
        #Add words and tag tuple to our xy list
        xy.append((words,tag))
    
    for response in intent['responses']:
        response_list.append(response)
    responses.append(response_list)
        
#Add tag and tag tuple to our tag_idx list
for idx in range(len(tags)):
    tag_idx.append((idx, tags[idx]))


In [7]:
#Remove punctuation characters and transform to lower case
vocabulary = [stem(w.lower()) for w in vocabulary if w not in string.punctuation]

#Remove duplicated values
vocabulary = sorted(set(vocabulary))
tags = sorted(set(tags))

#### Comunidades autónomas

In [8]:
#Create an empty list with pattern words 
vCA = []
#Create an empty list with tag words 
tagsCA = []
#Create an empty list with tag and pattern words 
xyCA = []
#Create an empty list with tag and tag index 
tag_idxCA = []
#Create empty list with telephones
tlf = []

for cam in ca['CA']:
    tag = cam['coma']
    tagsCA.append(tag)
    
    for pattern in cam['patterns']:
        #Tokenize each word in the sentence
        words = tokenize(pattern)        
        #Add words to the pattern_words list
        vCA.extend(words)
        #Add words and tag tuple to our xy list
        xyCA.append((words,tag))
    
    for telephone in cam['telephone']:
        tlf.append((cam["coma"], telephone))
        
        
#Add tag and tag tuple to our tag_idx list
for idx in range(len(tagsCA)):
    tag_idxCA.append((idx, tagsCA[idx]))
    
#Remove punctuation characters and transform to lower case
vCA = [w.lower() for w in vCA if w not in string.punctuation]

#Remove duplicated values
vCA = sorted(set(vCA))
tagsCA = sorted(set(tagsCA))

In [9]:
#Create empty lists for our training data 
X_trainCA = []
y_trainCA = []

"""
For each pattern sentence, we will create a bag of words and append it
to the X_train dataset, then we will append the output label for that 
sentence to the y_train dataset. This way, we will create a our training
dataset. 

The X_train dataset will consist of a matrix containing the bag vector of
each pattern sentence of 0's and 1's. On the other hand, the y_train will
consist of the output labels in the form of indexes, so the indexes will 
range from 0 to the length of the list "tags". 

"""
for (pattern_sentence, tag) in xyCA:
    bag = bag_nostem(pattern_sentence, vCA)
    X_trainCA.append(bag)
    
    label = tagsCA.index(tag)
    y_trainCA.append(label)
    
X_trainCA = np.array(X_trainCA)
y_trainCA = np.array(y_trainCA)

y_arrayCA = np.zeros((X_trainCA.shape[0], len(tagsCA)))

count = 0

for idx in range(len(y_trainCA)):
    label = y_trainCA[idx]
    y_arrayCA[count][label] = 1
    count += 1

In [10]:
# Neural network
modelCA = Sequential()
modelCA.add(Dense(16, input_dim = X_trainCA.shape[1], activation='relu'))
modelCA.add(Dense(12, activation='relu'))
modelCA.add(Dense(y_arrayCA.shape[1], activation='softmax'))

In [11]:
modelCA.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
modelCA.fit(X_trainCA, y_arrayCA, epochs=200, batch_size=10)

Epoch 1/200
37/37 [==============================] - 1s 20ms/step - loss: 2.9440 - accuracy: 0.0270
Epoch 2/200
37/37 [==============================] - 0s 315us/step - loss: 2.9348 - accuracy: 0.0270
Epoch 3/200
37/37 [==============================] - 0s 311us/step - loss: 2.9274 - accuracy: 0.0270
Epoch 4/200
37/37 [==============================] - 0s 333us/step - loss: 2.9207 - accuracy: 0.0541
Epoch 5/200
37/37 [==============================] - 0s 361us/step - loss: 2.9132 - accuracy: 0.0541
Epoch 6/200
37/37 [==============================] - 0s 373us/step - loss: 2.9063 - accuracy: 0.0541
Epoch 7/200
37/37 [==============================] - 0s 353us/step - loss: 2.8992 - accuracy: 0.1081
Epoch 8/200
37/37 [==============================] - 0s 325us/step - loss: 2.8919 - accuracy: 0.1081
Epoch 9/200
37/37 [==============================] - 0s 308us/step - loss: 2.8844 - accuracy: 0.0811
Epoch 10/200
37/37 [==============================] - 0s 321us/step - loss: 2.8765 - accurac

37/37 [==============================] - 0s 313us/step - loss: 0.2754 - accuracy: 1.0000
Epoch 161/200
37/37 [==============================] - 0s 279us/step - loss: 0.2683 - accuracy: 1.0000
Epoch 162/200
37/37 [==============================] - 0s 305us/step - loss: 0.2616 - accuracy: 1.0000
Epoch 163/200
37/37 [==============================] - 0s 322us/step - loss: 0.2548 - accuracy: 1.0000
Epoch 164/200
37/37 [==============================] - 0s 315us/step - loss: 0.2488 - accuracy: 1.0000
Epoch 165/200
37/37 [==============================] - 0s 310us/step - loss: 0.2423 - accuracy: 1.0000
Epoch 166/200
37/37 [==============================] - 0s 308us/step - loss: 0.2359 - accuracy: 1.0000
Epoch 167/200
37/37 [==============================] - 0s 303us/step - loss: 0.2304 - accuracy: 1.0000
Epoch 168/200
37/37 [==============================] - 0s 315us/step - loss: 0.2245 - accuracy: 1.0000
Epoch 169/200
37/37 [==============================] - 0s 302us/step - loss: 0.2189 - a

#### 2. Create vectors for training set

This data structure won't work with TensorFlow, which we will use for the neural network model, so we need to transform it further: from sets of words into tensors/arrays of numbers.

In [12]:
#Create empty lists for our training data 
X_train = []
y_train = []

"""
For each pattern sentence, we will create a bag of words and append it
to the X_train dataset, then we will append the output label for that 
sentence to the y_train dataset. This way, we will create a our training
dataset. 

The X_train dataset will consist of a matrix containing the bag vector of
each pattern sentence of 0's and 1's. On the other hand, the y_train will
consist of the output labels in the form of indexes, so the indexes will 
range from 0 to the length of the list "tags". 

"""
for (pattern_sentence, tag) in xy:
    bag = bag_of_words(pattern_sentence, vocabulary)
    X_train.append(bag)
    
    label = tags.index(tag)
    y_train.append(label)
    
X_train = np.array(X_train)
y_train = np.array(y_train)

y_array = np.zeros((X_train.shape[0], len(tags)))

count = 0

for idx in range(len(y_train)):
    label = y_train[idx]
    y_array[count][label] = 1
    count += 1

#### 3. Train the Neural Network Model

We will train our model with Keras. Keras is a recommended library for deep learning in Python. Its minimalistic, modular approach makes it a breeze to get deep in neural networks. 

Deep learning refers to neural networks with multiple hidden layers that can learn increasingly abstract representations of the input data. The higher the number of layers, the more complex the patterns the model learn and the more accurate the classification is. 


Models with Keras are defined as a sequence of layers. We create a Sequential model and add layers one at a time until we are happy with our network architecture.

In [13]:
from sklearn.model_selection import train_test_split

X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(X_train,y_array,test_size = 0.1, random_state = 6)

In [14]:
X_TRAIN.shape[1]

152

In [15]:
Y_TRAIN.shape

(146, 18)

In [16]:
# Neural network
model = Sequential()
model.add(Dense(16, input_dim = X_train.shape[1], activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(y_array.shape[1], activation='softmax'))

In [17]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
model.fit(X_train, y_array, epochs=200, batch_size=10)

Epoch 1/200
163/163 [==============================] - 1s 4ms/step - loss: 2.8951 - accuracy: 0.0613
Epoch 2/200
163/163 [==============================] - 0s 283us/step - loss: 2.8627 - accuracy: 0.1350
Epoch 3/200
163/163 [==============================] - 0s 265us/step - loss: 2.8348 - accuracy: 0.1963
Epoch 4/200
163/163 [==============================] - 0s 270us/step - loss: 2.8037 - accuracy: 0.2393
Epoch 5/200
163/163 [==============================] - 0s 262us/step - loss: 2.7666 - accuracy: 0.2577
Epoch 6/200
163/163 [==============================] - 0s 260us/step - loss: 2.7201 - accuracy: 0.2638
Epoch 7/200
163/163 [==============================] - 0s 263us/step - loss: 2.6619 - accuracy: 0.2515
Epoch 8/200
163/163 [==============================] - 0s 266us/step - loss: 2.5920 - accuracy: 0.2209
Epoch 9/200
163/163 [==============================] - 0s 252us/step - loss: 2.5067 - accuracy: 0.2209
Epoch 10/200
163/163 [==============================] - 0s 295us/step - los

163/163 [==============================] - 0s 262us/step - loss: 0.0074 - accuracy: 1.0000
Epoch 158/200
163/163 [==============================] - 0s 272us/step - loss: 0.0073 - accuracy: 1.0000
Epoch 159/200
163/163 [==============================] - 0s 263us/step - loss: 0.0071 - accuracy: 1.0000
Epoch 160/200
163/163 [==============================] - 0s 264us/step - loss: 0.0070 - accuracy: 1.0000
Epoch 161/200
163/163 [==============================] - 0s 254us/step - loss: 0.0068 - accuracy: 1.0000
Epoch 162/200
163/163 [==============================] - 0s 258us/step - loss: 0.0067 - accuracy: 1.0000
Epoch 163/200
163/163 [==============================] - 0s 271us/step - loss: 0.0066 - accuracy: 1.0000
Epoch 164/200
163/163 [==============================] - 0s 280us/step - loss: 0.0064 - accuracy: 1.0000
Epoch 165/200
163/163 [==============================] - 0s 280us/step - loss: 0.0063 - accuracy: 1.0000
Epoch 166/200
163/163 [==============================] - 0s 285us/ste

In [ ]:
print(colored("Hi! My name is Cova", attrs = ["bold"]) , "\U0001F916",
      colored("\n\nI have been created to try to help you with all your possible questions related to Covid-19.\nI am delighted to speak with you and help you in any way I can.",attrs = ["bold"]),
      colored("\n\nI can give you information about many things!", attrs = ["bold"]), 
      "\n\n\U00002705", colored("What is the coronavirus?", attrs = ["bold"]), "\U0001F9A0",
      "\n\U00002705", colored("Covid phone", attrs = ["bold"]),"\U0001F4DE",
      "\n\U00002705", colored("Symptoms", attrs = ["bold"]),"\U0001F927\U0001F321",      
      "\n\U00002705", colored("Contact with positives", attrs = ["bold"]),"\U0001F91D\U000026A0",
      "\n\U00002705", colored("Quarantine", attrs = ["bold"]),"\U0001F3E0", 
      "\n\U00002705", colored("Tips to protect yourself from the virus", attrs = ["bold"]),"\U0001F44F\U0001F9FC\U00002728",
      colored("\n\nCome on! Ask me something", attrs =["bold"]), "\U0001F601"

     )

conversation = True


while conversation:
    
    print("")
    #Ask for a question from the user
    response = input()
    
    #Response without grammar parser
    response_p = tokenize(response)
    
    #Grammar correction of the question
    parser = GingerIt()
    result = parser.parse(response)
    response = result['result']
    
    #Tokenization and bag of words
    x_sentence = tokenize(response)
    bag_x = bag_of_words(x_sentence, vocabulary)
    bag_test = np.zeros((1, X_train.shape[1]))
    bag_test[0] = bag_x
    
    #Prediction of the response
    predictions = model.predict_classes(bag_test)
    probability = model.predict_proba(bag_test)
    
    #for element in probability[0]:
    #    print(element)
    
    if probability[0][predictions[0]] >= 0.85:
    
        label_response = tags[predictions[0]]

        for idx, element in tag_idx:
            
            if element == label_response:
                print("")
                output = colored(random.choice(responses[idx]).encode('ascii').decode('unicode-escape'), attrs = ["bold"])
                print(output)
                
                if label_response == 'CovidTelephone':
                    
                    #Bag of words for Autonomous Community
                    bag_ca = bag_nostem(response_p, vCA)
                    bag_testca = np.zeros((1, X_trainCA.shape[1]))
                    bag_testca[0] = bag_ca
                    
                    #Predictions for Autonomous Community
                    prediction_ca = modelCA.predict_classes(bag_testca)
                    probability_ca = modelCA.predict_proba(bag_testca)
                    
                    if probability_ca[0][prediction_ca][0] >= 0.25:
                        community = tagsCA[prediction_ca[0]]
                        
                        for com, phone in tlf: 
                            if com == community:
                                phone_number = phone
                            
                        response_tlf = "\n\nThe telephone number of {} is: {}".format(community, phone_number)
                        print(colored(response_tlf, attrs = ["bold"]))
                    
                    else:
                        print(colored("\nI can help you with Covid-19 phone numbers. What is your autonomous community?\n", attrs=["bold"]))
                        
                        input_ca = input()

                        #Tokenize input and bag of words
                        input_ca = tokenize(input_ca)
                        bag_caq = bag_nostem(input_ca, vCA)
                        bag_testcaq = np.zeros((1, X_trainCA.shape[1]))
                        bag_testcaq[0] = bag_caq


                        #Predictions for Autonomous Community
                        prediction_caq = modelCA.predict_classes(bag_testcaq)
                        probability_caq = modelCA.predict_proba(bag_testcaq)
                        if probability_caq[0][prediction_caq][0] >= 0.25:
                            community = tagsCA[prediction_caq[0]]

                            for com, phone in tlf: 
                                if com == community:
                                    phone_number = phone

                            response_tlf = "\nThe telephone number of {} is: {}".format(community, phone_number)
                            print(colored(response_tlf, attrs = ["bold"]))
                        
                        else:
                            print(colored("\nIm sorry, I didn't understand your response", attrs = ["bold"]))
    else:
        print(colored("\nIm sorry, I didn't understand your question", attrs = ["bold"]))
        


Hi! My name is Cova 🤖 

I have been created to try to help you with all your possible questions related to Covid-19.
I am delighted to speak with you and help you in any way I can. 

I can give you information about many things! 

✅ What is the coronavirus? 🦠 
✅ Covid phone 📞 
✅ Symptoms 🤧🌡 
✅ Contact with positives 🤝⚠ 
✅ Quarantine 🏠 
✅ Tips to protect yourself from the virus 👏🧼✨ 

Come on! Ask me something 😁

covid phone la rioja

It's important to not to collapse emergency phone lines. If you only need information, you should call the Covid phone number provided by your autonomous community.

🆘☎ Just in case of emergency, call 112. 


The telephone number of La Rioja is: 941 298 333

covid phone andalucia

It's important to not to collapse emergency phone lines. If you only need information, you should call the Covid phone number provided by your autonomous community.

🆘☎ Just in case of emergency, call 112. 


The telephone number of Andalucia is: 900 400 061 / 955 545 060

give me